In [18]:
!gdown 18iEZmOkDB775R1dsfLIXJVqIHdkNpUSt

Downloading...
From: https://drive.google.com/uc?id=18iEZmOkDB775R1dsfLIXJVqIHdkNpUSt
To: /work/some/wiki_bot/exps/labse_vectorize_splitted_html/moodle.zip
100%|██████████████████████████████████████| 11.3M/11.3M [00:00<00:00, 70.4MB/s]


In [20]:
!mkdir data

In [ ]:
!unzip moodle.zip -d data

In [23]:
import pandas as pd
from parse_proc.split_page import main
from transformers import AutoTokenizer, AutoModelForPreTraining

In [24]:
index = pd.read_csv("data/1course/1_1/links_1_1.csv", sep=";", names=["title", "url", "filename"])

In [25]:
index.head()

,title,url,filename
0,Дополнительные задачи по программированию для ...,https://se.moevm.info/doku.php/courses:program...,1.html
1,Форма для записи результатов,https://forms.gle/RJySDieYjnYoBBKL8,2.html
2,Правила и описание рейтинговой системы на осен...,https://docs.google.com/document/d/e/2PACX-1vT...,3.html
3,Правила и описание рейтинговой системы на весн...,https://docs.google.com/document/d/e/2PACX-1vT...,4.html
4,Правила дистанционной защиты лабораторных и ку...,https://docs.google.com/document/d/e/2PACX-1vT...,5.html


In [26]:
def filter_urls(x):
    good_domain = [
        "docs.google.com",
        "se.moevm.info"
    ]
    
    for domain in good_domain:
        if domain in x:
            return 1
    
    return 0

In [27]:
index["is_good"] = index["url"].apply(filter_urls)
filtered_index = index[index['is_good'] == 1]
filtered_index = filtered_index.drop(columns="is_good")

In [28]:
filtered_index.to_csv("filtered_links_1_1.csv", sep=";", header=False)

In [29]:
from text_dataset_from_html import TextDatasetFromHTML
from search_space import SearchSpace
import torch
import torch.nn.functional as F

In [30]:
dataset = TextDatasetFromHTML(
    "data/1course/1_1/",
    "filtered_links_1_1.csv",
    split_func=main
    )

In [31]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModelForPreTraining.from_pretrained("cointegrated/LaBSE-en-ru")
model = model.to("cuda:0")

In [32]:
def vectorize_func(text):
    encoded_input = tokenizer([text], padding=True, return_tensors='pt')
    encoded_input.to(model.device)
    with torch.no_grad():
        model_output = model(**encoded_input, output_hidden_states=True)
    embeddings = model_output.hidden_states[-1]
    embeddings = torch.mean(embeddings, axis=1)
    embeddings = torch.nn.functional.normalize(embeddings)

    return embeddings.detach().cpu()

In [33]:
# can be more faster with batching or by using mesh/KD-Tree/etc
search_space = SearchSpace([dataset], text_vectorize_func=vectorize_func)

In [34]:
def cos_sim(a, b):
    return F.cosine_similarity(a, b)

In [35]:
dist, res = search_space.find_top_nearest("Сколько будет контрольных работ у 1 курса по программированию?", metric=cos_sim, cnt=2)

In [36]:
print(res[0][1])

Правила и описание рейтинговой системы на осень 2022


In [15]:
dist

tensor([0.6626, 0.6417])